# Análisis de Imágenes con Amazon Rekognition

Este notebook procesa imágenes almacenadas en S3 utilizando AWS Rekognition Custom Labels para detectar objetos y visualizar los resultados.

## 1. Configuración Inicial
Importación de bibliotecas necesarias y configuración de parámetros de AWS.

In [ ]:
import boto3
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json
from datetime import datetime

# Configuración global
CONFIG = {
    # Configuración AWS
    'SOURCE_BUCKET': '<SOURCE_BUCKET>',    # Nombre del bucket S3 que contiene los frames
    'SOURCE_PREFIX': 'frames/',             # Carpeta dentro del bucket donde están los frames
    'PROJECT_ARN': '<ARN_PROYECTO>',
    'MODEL_ARN': '<ARN_MODELO>',
    'AWS_REGION': '<REGION>',             # Región AWS
    'MIN_CONFIDENCE': 40,                  # Umbral de confianza para detecciones (%)
    'OUTPUT_JSON': 'resultados_detecciones.json'  # Archivo de salida
}

# Inicializar clientes de AWS
s3 = boto3.client('s3', region_name=CONFIG['AWS_REGION'])
rekognition = boto3.client('rekognition', region_name=CONFIG['AWS_REGION'])

## 2. Función Principal de Procesamiento
Esta función procesa una imagen individual, realiza la detección con Rekognition y muestra los resultados.

In [19]:
def process_and_display_image(bucket, key, show_image=False):
    """
    Procesa una imagen y opcionalmente la muestra
    """
    try:
        # Descargar la imagen de S3
        response = s3.get_object(Bucket=bucket, Key=key)
        image_bytes = response['Body'].read()
        
        # Convertir bytes a numpy array
        nparr = np.frombuffer(image_bytes, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Llamar a Rekognition Custom Labels
        response = rekognition.detect_custom_labels(
            Image={'S3Object': {'Bucket': bucket, 'Name': key}},
            ProjectVersionArn=CONFIG['MODEL_ARN'],
            MinConfidence=CONFIG['MIN_CONFIDENCE']
        )
        
        frame_name = key.split('/')[-1]
        result_dict = {
            'frame': frame_name,
            'detections': [{
                'label': label['Name'],
                'confidence': round(label['Confidence'], 2)
            } for label in response['CustomLabels']]
        }
        
        # Mostrar imagen solo si show_image es True
        if show_image:
            plt.figure(figsize=(12, 8))
            plt.imshow(image)
            plt.axis('off')
            if response['CustomLabels']:
                plt.title(f"Detección en: {frame_name}\nConfianza: {response['CustomLabels'][0]['Confidence']:.2f}%")
            else:
                plt.title(f"Sin detecciones en: {frame_name}")
            plt.show()
        
        return result_dict, bool(response['CustomLabels']), image
    
    except Exception as e:
        print(f"Error procesando la imagen {key}: {str(e)}")
        return None, None, None


## 3. Función de Listado de Objetos S3
Función auxiliar para listar todos los objetos en el bucket S3.

In [20]:
def list_s3_objects(bucket, prefix):
    try:
        paginator = s3.get_paginator('list_objects_v2')
        for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
            for obj in page.get('Contents', []):
                yield obj['Key']
    except Exception as e:
        print(f"Error al listar objetos en {bucket}/{prefix}: {str(e)}")

## 4. Ejecución del Procesamiento
Procesa todas las imágenes en el bucket S3 especificado.

In [ ]:
def main():
    print("Iniciando procesamiento de imágenes...")
    
    all_results = []
    detected_examples = []
    non_detected_examples = []
    
    # Procesar imágenes hasta encontrar suficientes ejemplos
    for key in list_s3_objects(CONFIG['SOURCE_BUCKET'], CONFIG['SOURCE_PREFIX']):
        if not key.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
            
        result, has_detection, image = process_and_display_image(CONFIG['SOURCE_BUCKET'], key, False)
        
        if result:
            all_results.append(result)
            
            # Guardar ejemplos según si tienen detección o no
            if has_detection and len(detected_examples) < 2:
                detected_examples.append((key, image))
            elif not has_detection and len(non_detected_examples) < 2:
                non_detected_examples.append((key, image))
                
        # Salir si tenemos suficientes ejemplos
        if len(detected_examples) == 2 and len(non_detected_examples) == 2:
            break
    
    # Mostrar los ejemplos
    print("\nEjemplos con detección de logo:")
    for key, image in detected_examples:
        process_and_display_image(CONFIG['SOURCE_BUCKET'], key, True)
        
    print("\nEjemplos sin detección de logo:")
    for key, image in non_detected_examples:
        process_and_display_image(CONFIG['SOURCE_BUCKET'], key, True)
    
    # Guardar resultados
    try:
        with open(CONFIG['OUTPUT_JSON'], 'w') as f:
            json.dump(all_results, f, indent=2)
        print(f"\nTodos los resultados guardados en: {CONFIG['OUTPUT_JSON']}")
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")

if __name__ == "__main__":
    main()

## Limpieza de Recursos AWS

Esta sección elimina todos los recursos AWS creados en este proyecto.

In [ ]:
def cleanup_aws_resources():
    """Limpia todos los recursos AWS creados"""
    
    try:
        # 1. Detener el modelo de Rekognition
        print("Deteniendo modelo de Rekognition...")
        rekognition.stop_project_version(ProjectVersionArn=CONFIG['MODEL_ARN'])
        print("✓ Modelo detenido")
        
        # 2. Eliminar proyecto de Rekognition
        print("Eliminando proyecto de Rekognition...")
        rekognition.delete_project(ProjectArn=CONFIG['PROJECT_ARN'])
        print("✓ Proyecto eliminado")
        
        # 3. Vaciar y eliminar buckets S3
        buckets_to_clean = [CONFIG['SOURCE_BUCKET']]
        
        for bucket_name in buckets_to_clean:
            try:
                print(f"Limpiando bucket {bucket_name}...")
                
                # Listar y eliminar todos los objetos
                response = s3.list_objects_v2(Bucket=bucket_name)
                if 'Contents' in response:
                    objects = [{'Key': obj['Key']} for obj in response['Contents']]
                    s3.delete_objects(Bucket=bucket_name, Delete={'Objects': objects})
                    print(f"✓ {len(objects)} objetos eliminados")
                
                # Eliminar el bucket
                s3.delete_bucket(Bucket=bucket_name)
                print(f"✓ Bucket {bucket_name} eliminado")
                
            except Exception as e:
                print(f"⚠️ Error limpiando bucket {bucket_name}: {e}")
        
        print("\n🎉 Limpieza completada exitosamente")
        
    except Exception as e:
        print(f"❌ Error durante la limpieza: {e}")

# Esto eliminará TODOS los recursos. Ejecutar solo cuando esté seguro.

## Notas Finales
- El código procesa imágenes almacenadas en S3
- Utiliza AWS Rekognition Custom Labels para la detección
- Muestra las imágenes originales
- Proporciona información sobre las detecciones realizadas
- Elimina los recursos desplegados